In [80]:
import pandas as pd
import string
from nltk.corpus import stopwords

In [81]:
#Get the spam data collection 
df_spam=pd.read_csv('SpamCollection',sep='\t',names=['response','message'])

In [82]:
df_spam.head()

,response,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [83]:
df_spam.describe()
df_spam.shape

(5572, 2)

In [84]:
#view response 
df_spam.groupby('response').describe()

message                                                            \
           count unique                                                top   
response                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
response       
ham        30  
spam        4

In [85]:
#Verify length of the messages and also add it as a new column 
df_spam['length']=df_spam['message'].apply(len)

In [86]:
df_spam.head()

,response,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [87]:
#define a function to get rid of stopwords present in the messages
def message_test(mess):
    no_pun=[char for char in mess if char not in string.punctuation]
    no_pun=''.join(no_pun)
    return[word for word in no_pun.split() if word.lower() not in stopwords.words('english')]

In [88]:
df_spam['message'].head().apply(message_test)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: message, dtype: object

In [103]:
#start text processing with vectorizer 
from sklearn.feature_extraction.text import CountVectorizer

In [104]:
#use bag of words by applying the function and fit the data into it
bag_of_word=CountVectorizer(analyzer=message_test).fit(df_spam['message'])

In [105]:
#print length of bag of words stored in the vocabulary_ attribute
print(len(bag_of_word.vocabulary_))

11425


In [122]:
message_ba_of_word=bag_of_word.transform(df_spam['message'])
print(message_ba_of_word)

  (0, 1110)	1
  (0, 1483)	1
  (0, 2060)	1
  (0, 4653)	1
  (0, 5217)	1
  (0, 5218)	1
  (0, 5769)	1
  (0, 6217)	1
  (0, 6906)	1
  (0, 6937)	1
  (0, 7555)	1
  (0, 7668)	1
  (0, 8336)	1
  (0, 8917)	1
  (0, 10965)	1
  (0, 11163)	1
  (1, 2451)	1
  (1, 3064)	1
  (1, 7701)	1
  (1, 8590)	1
  (1, 10698)	1
  (1, 11072)	1
  (2, 73)	1
  (2, 423)	1
  (2, 430)	1
  :	:
  (5568, 6691)	1
  (5568, 6882)	1
  (5568, 7159)	1
  (5568, 11418)	1
  (5569, 3228)	1
  (5569, 3721)	1
  (5569, 8252)	1
  (5569, 10199)	1
  (5570, 4508)	1
  (5570, 5055)	1
  (5570, 5251)	1
  (5570, 6282)	1
  (5570, 6699)	1
  (5570, 6799)	1
  (5570, 6984)	1
  (5570, 7287)	1
  (5570, 7394)	1
  (5570, 7800)	1
  (5570, 8420)	1
  (5570, 9915)	1
  (5570, 10787)	1
  (5570, 11006)	1
  (5571, 3431)	1
  (5571, 8348)	1
  (5571, 10648)	1


In [107]:
#apply tfidf transformer and fit the bag of words into it (transformed version)
from sklearn.feature_extraction.text import TfidfTransformer

In [108]:
#print shape of the tfidf 
tfidf=TfidfTransformer().fit(message_ba_of_word)
mess_tfid=tfidf.transform(message_ba_of_word)
mess_tfid.shape

(5572, 11425)

In [109]:
#choose naive Bayes model to detect the spam and fit the tfidf data into it
from sklearn.naive_bayes import MultinomialNB
spam_detect=MultinomialNB().fit(mess_tfid,df_spam['response'])

In [116]:
#check model for the predicted and expected value say for message#2 and message#5
message=df_spam['message'][10]
bag_of_word1=bag_of_word.transform([message])

In [121]:
tfidf1=tfidf.transform(bag_of_word1)
print('predicted: ',spam_detect.predict(tfidf1)[0])
print('expected: ',df_spam.response[10])

predicted:  ham
expected:  ham
